Medical Graph Database: Importing Cases, Symptoms, and Diagnoses into Neo4j

In [1]:
# Import necessary libraries
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [2]:
# Retrieve Neo4j connection details from environment variables
NEO4J_URI = os.getenv('NEO4J_URI')                      # Neo4j database URI
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')            # Username for authentication
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')            # Password for authentication
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')            # Database name to connect to
MEDICAL_CASES_ROUTE = os.getenv('MEDICAL_CASES_ROUTE')  # Route to the csv-file

In [3]:
# Initialize Neo4jGraph with connection details for database access
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

In [4]:
graph.query("""
LOAD CSV WITH HEADERS FROM 'file:///' + $movie_directory AS row

// Erstellen des Falls und der Person
MERGE (f:Fall {id:row.Fall_ID})                                  
MERGE (p:Person {id:row.Person_ID})
SET p.alter = row.Alter,
    p.adresse = row.Adresse,
    p.geschlecht = row.Geschlecht
MERGE (f)-[:FÜR]->(p)

// Erstellen der Symptome und deren Beziehung zur Person
FOREACH (Symptom IN split(row.Symptom, '|') |             
    MERGE (s:Symptom {name:trim(Symptom)})                  
    MERGE (p)-[:HAT_SYMPTOM]->(s))

// Erstellen der Diagnosen und deren Beziehung zur Person
FOREACH (Diagnose IN split(row.Diagnose, '|') |             
    MERGE (d:Diagnose {name:trim(Diagnose)})                  
    MERGE (p)-[:HAT_DIAGNOSE]->(d))

// Erstellen der Krankheiten und deren Beziehung zur Person
FOREACH (Krankheit IN split(row.Krankheit, '|') |             
    MERGE (k:Krankheit {name:trim(Krankheit)})                  
    MERGE (p)-[:HAT_KRANKHEIT]->(k))
            
// Erstellen der Biologisch Knoten
FOREACH (Biologisch IN split(row.Biologisch, '|') |             
    MERGE (b:Biologisch {name:trim(Biologisch)})) 
            
// Erstellen der Körpersystemknoten
FOREACH (Körpersystem IN split(row.Körpersystem, '|') |             
    MERGE (ks:Körpersystem {name:trim(Körpersystem)})) 

// Erstellung der Diagnose-Krankheit-Beziehungen basierend auf Diagnose_Rel_Krankheit
WITH row, split(row.Diagnose, '|') AS diagnosen, split(row.Krankheit, '|') AS krankheiten,
     [rel in split(row.Diagnose_Rel_Krankheit, '|') | split(rel, ',')] AS rel_matrix

// Äußere Schleife über die Diagnosen
FOREACH (i IN range(0, size(diagnosen)-1) |
    // Innere Schleife über die Krankheiten
    FOREACH (j IN range(0, size(krankheiten)-1) |
        // Prüfen, ob die Beziehung vorhanden ist (Wert = '1')
        FOREACH (_ IN CASE WHEN rel_matrix[i][j] = '1' THEN [1] ELSE [] END |
            MERGE (d:Diagnose {name: trim(diagnosen[i])})
            MERGE (k:Krankheit {name: trim(krankheiten[j])})
            MERGE (d)-[:ZEIGT]->(k)
        )
   )
)
            
// Erstellung der Diagnose-Biologisch-Beziehungen basierend auf Diagnose_Rel_Biologisch
WITH row, split(row.Diagnose, '|') AS diagnosen, split(row.Biologisch, '|') AS biologisch,
     [rel in split(row.Diagnose_Rel_Biologisch, '|') | split(rel, ',')] AS rel_matrix

// Äußere Schleife über die Diagnosen
FOREACH (i IN range(0, size(diagnosen)-1) |
    // Innere Schleife über Biologisch
    FOREACH (j IN range(0, size(biologisch)-1) |
        // Prüfen, ob die Beziehung vorhanden ist (Wert = '1')
        FOREACH (_ IN CASE WHEN rel_matrix[i][j] = '1' THEN [1] ELSE [] END |
            MERGE (d:Diagnose {name: trim(diagnosen[i])})
            MERGE (b:Biologisch {name: trim(biologisch[j])})
            MERGE (d)-[:ZEIGT]->(b)
        )
    )
)
            
// Erstellung der Krankheit-Körpersystem-Beziehungen basierend auf Krankheit_Rel_Körpersystem
WITH row, split(row.Krankheit, '|') AS krankheiten, split(row.Körpersystem, '|') AS körpersysteme,
     [rel in split(row.Krankheit_Rel_Körpersystem, '|') | split(rel, ',')] AS rel_matrix

// Äußere Schleife über die Krankheiten
FOREACH (i IN range(0, size(krankheiten)-1) |
    // Innere Schleife über Körpersysteme
    FOREACH (j IN range(0, size(körpersysteme)-1) |
        // Prüfen, ob die Beziehung vorhanden ist (Wert = '1')
        FOREACH (_ IN CASE WHEN rel_matrix[i][j] = '1' THEN [1] ELSE [] END |
            MERGE (k:Krankheit {name: trim(krankheiten[i])})
            MERGE (ks:Körpersystem {name: trim(körpersysteme[j])})
            MERGE (k)-[:BETRIFFT]->(ks)
        )
    )
)
""",
params={"movie_directory": str(MEDICAL_CASES_ROUTE)})

[]